In [1]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [2]:
# Import Libraries
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import dgl
import torch
import random
from dgl import save_graphs, load_graphs

/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL/.venv/lib/python3.12/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
import dgl

In [4]:
pwd

'/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL'

In [5]:
path = "./data2"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [6]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [7]:
train_data.dataset

Dataset("GraphDGL", num_graphs=200, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [8]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [9]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [10]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

In [11]:
n_features = 5
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=14)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    for i, graph in enumerate(dataset.graphs):
        node_features = graph.ndata['feat']
        node_labels = graph.ndata['label']
        train_mask = graph.ndata['train_mask']
        valid_mask = graph.ndata['val_mask']
        test_mask = graph.ndata['test_mask']
        n_features = node_features.shape[1]
        n_labels = int(node_labels.max().item() + 1)
        
        model.train()
        # forward propagation by using all nodes
        logits = model(graph, node_features)
        # compute loss
        loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
        # compute validation accuracy
        acc = evaluate(model, graph, node_features, node_labels, valid_mask)
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    
        # Save model if necessary.  Omitted in this example.

33.032413482666016
29.924386978149414
27.556358337402344
26.91301155090332
19.778169631958008
16.176483154296875
15.53268814086914
15.228702545166016
16.424402236938477
12.51594066619873
10.749863624572754
8.245844841003418
3.8372745513916016
8.71058177947998
9.916980743408203
6.998727321624756
7.576217174530029
7.770593643188477
7.100915431976318
6.285816669464111
5.637589454650879
4.782248497009277
5.252902984619141
4.404629230499268
3.527855157852173
3.234246015548706
2.152468204498291
1.7303928136825562
1.385141134262085
2.9713082313537598
2.400442600250244
2.8796632289886475
3.7223777770996094
3.5037264823913574
3.091165542602539
3.091073513031006
2.858088493347168
2.7741293907165527
2.349722385406494
2.2652106285095215
1.8242007493972778
2.2013158798217773
1.9658782482147217
2.2780237197875977
1.4048584699630737
2.344586133956909
1.524590015411377
1.5597748756408691
1.8706783056259155
1.6454464197158813
1.418070912361145
1.3503267765045166
1.3127596378326416
1.4516347646713257
1.

In [12]:
DGL.ModelClassifyNodes(model, test_data.dataset)

{'alllabels': [[13,
   13,
   12,
   12,
   11,
   12,
   13,
   12,
   2,
   12,
   13,
   13,
   0,
   13,
   13,
   3,
   13,
   7,
   4,
   12,
   1,
   12,
   12,
   13,
   8,
   12,
   13,
   6,
   9,
   13],
  [12,
   12,
   6,
   12,
   13,
   13,
   9,
   2,
   4,
   7,
   6,
   13,
   3,
   8,
   0,
   1,
   13,
   13,
   13,
   11,
   13,
   13,
   12,
   13,
   12,
   12,
   13,
   7,
   13,
   13,
   12,
   13,
   12],
  [12,
   13,
   1,
   12,
   11,
   13,
   13,
   13,
   0,
   4,
   12,
   13,
   13,
   12,
   13,
   12,
   6,
   12,
   13,
   7,
   13,
   7,
   12,
   13,
   12,
   8,
   6,
   9,
   13,
   3,
   2,
   13,
   13],
  [13,
   12,
   1,
   13,
   12,
   0,
   12,
   13,
   13,
   13,
   12,
   12,
   13,
   13,
   13,
   13,
   8,
   3,
   13,
   6,
   13,
   4,
   13,
   12,
   2,
   6,
   7,
   12,
   12,
   13,
   7,
   11,
   9],
  [0,
   13,
   12,
   12,
   13,
   12,
   7,
   13,
   13,
   12,
   12,
   13,
   13,
   4,
   13,
   13,
   13,
   12,

In [13]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
# print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)
# print(predicted)
result = DGL.Accuracy(actual=actual, predicted=predicted)
# print(result)
print("Accuracy:", result['accuracy'])

Accuracy: 0.882436


In [14]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 150    0    0    0    0    0    0    0    0   15    0    0    0    0]
 [   0   48    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0   40    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0   40    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0  198  197    0   68  392    0    0    6    0   40    0    0]
 [   0    0    0    0   90    0    0  445  143    0    0    0    0    0]
 [   0    0    0    0   35    0    0    0   27    0    0   25    0    0]
 [   0    0    0    0   35    0    0    0    0  106    0    0    0    0]
 [   0   82    0    0    0    0    0    0    0   27  230    0    0    0]
 [   0    0    2    3    0    2    0    0   30    0    0  159    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0 3078    0]
 [   0    0    0    0    0    0    0    0    0    0